# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [17]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [18]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [63]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE

    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    try:
        data = pickle.load(open(filename, 'rb'))
        target = [i] * len(data)
    except ValueError as e:

        pose = 0
        data.append([])
        
        with open(filename, 'r') as file:
            lines = file.readlines()
            for line in lines:
                
                if line.startswith('F'):
                    values = line.strip().split('F')[1:]
                    values = [float(value) for value in values][0]
                    
                    if (len(data[pose]) < 10):
                        data[pose].append(values)
                    else:
                        pose = pose + 1
                        data.append([])
                        data[pose].append(values)

                if line.startswith('aF'):
                    values = line.strip().split('aF')[1:]
                    values = [float(value) for value in values][0]
                    
                    if (len(data[pose]) < 10):
                        data[pose].append(values)
                    else:
                        pose = pose + 1
                        data.append([])
                        data[pose].append(values)

                if line.startswith('I'):
                    values = line.strip().split('I')[1:]
                    values = [float(value) for value in values][0]
                    
                    if (len(data[pose]) < 10):
                        data[pose].append(values)
                    else:
                        pose = pose + 1
                        data.append([])
                        data[pose].append(values)

                if line.startswith('aI'):
                    values = line.strip().split('aI')[1:]
                    values = [float(value) for value in values][0]
                    
                    if (len(data[pose]) < 10):
                        data[pose].append(values)
                    else:
                        pose = pose + 1
                        data.append([])
                        data[pose].append(values)


        
                
        target = [i] * len(data)
        
            
    #filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    #data = pickle.load(open(filename, 'rb'))
    #target = [i] * len(data)
    return data, target

In [90]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE

for i in range (len(classes)):
    data, target = load_pose_data(i)
    for j in range (len(data)):
        all_data.append(data[j])
        all_target.append(target[j])
    #all_target.append(target[0])
        

print ('total number of data', len(all_data))

total number of data 204


In [91]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [92]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [93]:
# YOUR CODE HERE
clf.fit(all_data[:-102], all_target[:-102]) 

SVC(C=100.0, gamma=0.001)

### predicting

In [98]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([2]), [10])

In [99]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [101]:
expected = []
predicted = []
# YOUR CODE HERE
predicted = clf.predict(all_data[-102:])
evaluate(all_target[-102:], predicted)

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           6       0.09      1.00      0.16         2
           7       0.00      0.00      0.00        11
           8       0.00      0.00      0.00        26
           9       0.00      0.00      0.00        11
          10       0.00      0.00      0.00        52

    accuracy                           0.02       102
   macro avg       0.01      0.14      0.02       102
weighted avg       0.00      0.02      0.00       102


Confusion matrix:
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  2  0  0  0  0]
 [11  0  0  0  0  0  0]
 [ 0 18  8  0  0  0  0]
 [ 0  0 11  0  0  0  0]
 [50  0  2  0  0  0  0]]


C:\Users\Duc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Duc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Duc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{met

ValueError: Found input variables with inconsistent numbers of samples: [0, 102]

## 4. Evaluate on the test data

In [102]:
expected = []
predicted = []
# YOUR CODE HERE

predicted = clf.predict(all_data[-102:])
evaluate(all_target[-102:], predicted)

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           6       0.09      1.00      0.16         2
           7       0.00      0.00      0.00        11
           8       0.00      0.00      0.00        26
           9       0.00      0.00      0.00        11
          10       0.00      0.00      0.00        52

    accuracy                           0.02       102
   macro avg       0.01      0.14      0.02       102
weighted avg       0.00      0.02      0.00       102


Confusion matrix:
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  2  0  0  0  0]
 [11  0  0  0  0  0  0]
 [ 0 18  8  0  0  0  0]
 [ 0  0 11  0  0  0  0]
 [50  0  2  0  0  0  0]]


C:\Users\Duc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Duc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Duc\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{met

ValueError: Found input variables with inconsistent numbers of samples: [0, 102]

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [103]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [104]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 43: character maps to <undefined>